In [1]:
from flask import Flask, jsonify, request

app = Flask(__name__)

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

df = pd.read_csv('test.csv')


In [2]:
df

,Path,Question,Answer,Context
0,1.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
1,2.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
2,3.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
3,4.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
4,5.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
...,...,...,...,...
874,875.png,wine and beer are produced without whatdistill...,Wine-and-beer are produced without distillation.,Microbes especially yeast is being used from t...
875,876.png,wine is produced by fermentation of what by ye...,Wine is produced by fermentation of what by ye...,Microbes especially yeast is being used from t...
876,877.png,without what process is whisky obtainedferment...,"Without fermentation, the mango pulp is made i...",Microbes especially yeast is being used from t...
877,878.png,yeast and bacteria are used for commercial pro...,Yeast and bacteria are used for commercial pro...,Microbes are not only used for commercial and ...


In [3]:
df.head()

,Path,Question,Answer,Context
0,1.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
1,2.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
2,3.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
3,4.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
4,5.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."


In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model1 = SentenceTransformer('distilbert-base-nli-mean-tokens')


c:\Users\Rajesh\Desktop\Project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import gensim
from gensim import similarities
def sim(sentence):
    sentence = sentence
    sentences = list(df['Question'])
    dictionary = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(s) for s in sentences])

    tfidf_corpus = [dictionary.doc2bow(s) for s in [gensim.utils.simple_preprocess(s) for s in sentences]]
    tfidf_model = gensim.models.TfidfModel(tfidf_corpus)
    index = similarities.MatrixSimilarity(tfidf_model[tfidf_corpus], num_features=len(dictionary))

    query_vec = tfidf_model[dictionary.doc2bow(gensim.utils.simple_preprocess(sentence))]

    sims = index[query_vec]

    max_index = sims.argmax()
    return max_index

In [6]:
def similarity(question, lst):
    sentence_vector = model1.encode([question])[0]
    sentences_vectors = model1.encode(lst)

    from sklearn.metrics.pairwise import cosine_similarity
    similarities = [cosine_similarity([sentence_vector], [sentences_vector])[0][0] for sentences_vector in sentences_vectors]

    most_similar_index = similarities.index(max(similarities))
    return most_similar_index

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('model')
model = T5ForConditionalGeneration.from_pretrained('model')

In [8]:
def answer(question, context):

    input_text = f"Question: {question}\n\nContext: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output_ids = model.generate(input_ids,max_length=100)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True,)
    return output_text

In [9]:
def chat(question):
    input_text =  question 

    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    attention_mask = tokenizer(input_text, return_tensors='pt').attention_mask

    answer_ids = model.generate(input_ids, attention_mask=attention_mask,max_length=50)
    answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)
    return answer

In [10]:
def summarize(text):
    input_text = f"summarize the whole sentence into one line: {text} "
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output_ids = model.generate(input_ids,max_length=100)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True,)
    return output_text


In [11]:
def Question(question):

    i = sim(question)

    ans = answer(question,df['Context'][i])
    return [ans,df['Path'][i]]

In [12]:
def Ques(question):
    i = sim(question)
    ans = answer(question,df['Context'][i])
    return ans

In [13]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model2 = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer2 = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:

def translate(text):
    
    d = {}
    article_en = summarize(text)
    model_inputs = tokenizer2(article_en, return_tensors="pt")
    generated_tokens = model2.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer2.lang_code_to_id["ta_IN"],max_new_tokens = 1000
    )
    translation = tokenizer2.batch_decode(generated_tokens, skip_special_tokens=True)
    return translation[0]

In [16]:
@app.route('/api', methods=['GET'])
def api():
    d = {}
    q1 = str(request.args['text'])
    mode = q1[0]
    question = q1[1:]
    if mode == 'c' :
        answer= chat(question)
        d['answer'] = str(answer)
        d['path'] = str(answer)
        return d
    else:
        answer = Question(question)
        d['answer'] = str(answer[0])
        d['path'] = str(answer[1])
        return d
@app.route('/sum', methods=['GET'])
def sum():
    text = str(request.args['text'])
    sum = summarize(text)
    d = {}
    d['sum'] = str(sum)
    return d

@app.route('/trans', methods=['GET'])
def trans():
    d={}
    text = str(request.args['text'])
    d['trans']=translate(text)
    return d



In [20]:
import random
ls = []
i = -1
@app.route('/start', methods=['GET'])
def start():
    global ls
    if len(ls)!=0:
        
        ls = []
    id = str(request.args['text'])
    def get(id):
        ls.extend(random.sample(df1[id],10 if len(df1[id])>10 else len(df1
                                                                       [id])))
    get(id)
    return {"output": "success"}
    

In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [22]:
def evaluate(a,b):
    a1 = nlp(a)
    a2 = nlp(b)
    return a1.similarity(a2)

In [24]:
@app.route('/eval', methods=['GET'])
def eval():
    ans = str(request.args['text'])
    d={}
    if evaluate(ans,ls[i][1]) >=95:
        d['q'] = "Correct answer!!!"
    elif evaluate(ans,ls[i][1]) >=75:
        d['q'] = "You are close but the correct answer is \n" +Ques(ls[i][0])
    else:
        d['q'] = "Oops the answer is wrong!!! The answer is\n" +Ques(ls[i][0]) 
    return d

In [27]:
import re
from pdfminer.high_level import extract_text
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [28]:
@app.route('/pdf', methods=['GET'])

def pdf():

    ret={}
    pdf_path = str(request.args['text'])

    text = extract_text_from_pdf(pdf_path)

    text = text.replace('\u2018', '') 
    text = text.replace('\u2019', '') 
    text = text.replace('\u201C', '') 
    text = text.replace('\u201D', '') 

    lines = text.split('\n')

    data = {}
    current_heading = ''
    current_content = ''

    unwanted_patterns = [
        r'^\s*\[Page [0-9]+\]',  
    ]
    for line in lines:
        if any(re.match(pattern, line) for pattern in unwanted_patterns):
            continue

       
        if re.match(r'^[0-9]+\.[0-9]+ ', line): 
            if current_heading:
                data[current_heading] = current_content.strip()
            current_heading = line.strip()
            current_content = ''
        elif re.match(r'^[0-9]+\.[0-9]+\.[0-9]+ ', line): 
            if current_heading:
                data[current_heading] = current_content.strip()
            current_heading = line.strip()
            current_content = ''
        elif current_heading:
           
            current_content += ' ' + line.strip()
    if current_heading:
        data[current_heading] = current_content.strip()


    filtered_data = {k: v for k, v in data.items() if v is not None and v != ''}


    ret['out'] = filtered_data
    print("Output saved to 'outjson.json'")
    return ret

In [30]:
from pptx import Presentation
import json
import re
def extract_text_from_shape(shape):
    if shape.has_text_frame:
        text_parts = []
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                text_parts.append(run.text.strip())
        return ' '.join(text_parts)
    return ""

def extract_text_from_slide(slide):
    text = []
    for shape in slide.shapes:
        text_part = extract_text_from_shape(shape)
        if text_part:
            text.append(text_part)
    return ' '.join(text)


In [31]:
@app.route('/ppt', methods=['GET'])

def ppt():
    ret={}
    path2 = str(request.args['text'])
    presentation = Presentation(path2)

    material = {}

    current_topic = None

    for slide in presentation.slides:
        slide_info = {}
        slide_text = extract_text_from_slide(slide)
        if slide.shapes.title is not None:
            current_topic = slide.shapes.title.text.replace('\u000b', '').strip()
        if current_topic not in material:
            material[current_topic] = ""
        if slide_text.strip():
            slide_text = slide_text.replace(current_topic, "").strip()
            material[current_topic] += " " + slide_text.strip()

 
    for topic, content in material.items():
        content = re.sub(r'\s+', ' ', content)
        content = content.strip('. ')
        material[topic] = content
    ret['out'] = material
    print("Output saved to 'outjson.json'")
    return ret


In [32]:
@app.route('/ask', methods=['GET'])

def ask():
    global i 
    i = (i+1)%10
    d= {}
    d['q'] = ls[i][0]
    return d

In [33]:
@app.route('/img', methods=['GET'])
def img():
    paragraph =  str(request.args['text'])
    doc = nlp(paragraph)
    d = {}
    l=[]
    sentences = [sent.text for sent in doc.sents]
    for sentence in sentences:
        i = sim(sentence)

        l.append(df['Path'][i])
    d['img'] = l
    return d


In [34]:

if  __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 12:47:03] "GET /trans?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biological%20system%20on%20Earth.%20Several%20microorganisms%20arebeneficial%20and%20contribute%20to%20human%20welfare.%20Microbes%20are%20presenteverywhere%20–%20in%20soil,%20water,%20air%20and%20within%20bodies%20of%20animals%20and%20plants.%20Microbes%20like%20bacteria%20and%20fungi%20can%20be%20grown%20on%20nutritive%20media%20to%20form%20colonies%20which%20can%20be%20visibly%20seen.%20Some%20of%20the%20microbes%20useful%20to%20human%20welfare%20are%20discussed%20here HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:20:24] "GET /trans?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biologic

Output saved to 'outjson.json'


127.0.0.1 - - [01/Jun/2024 14:37:24] "GET /trans?text=Little%20Millet%20Botanical%20%20name-%20Panicum%20sumatrense%20%20This%20is%20one%20of%20the%20oldest%20millets%20and%20is%20native%20to%20India.%20The%20species%20name%20is%20based%20on%20a%20specimen%20collected%20from%20Sumatra.%20It%20is%20rich%20in%20iron%20and%20fibre%20than%20rice%20that%20makes%20it%20best%20for%20diabetes.%20%20Uses%20%20It%20is%20cooked%20like%20rice%20and%20also%20milled%20and%20baked.%20It%20%20cures%20%20anaemic%20%20condition,%20%20constipation%20%20and%20other%20gastrological%20problems.%20%20Foxtail%20Millet%20Botanical%20name%20:%20Setaria%20italica%20%20is%20%20one%20%20of%20%20This%20the%20%20oldest%20%20millet%20%20used%20traditionally%20%20in%20India.%20Which%20is%20domesticated%20first%20in%20China%20about%206000%20years.%20Rich%20in%20protein,%20carbohydrate,%20vitamin%20B%20and%20C,%20Potassium%20and%20Calcium.%20%20Uses%20It%20%20supports%20%20in%20%20strengthening%20%20of%20%20heart%20%20a

In [ ]:
import cv2
import os
import time
import dlib
from scipy.spatial import distance
import mediapipe as mp
from open_cv import HandTrackingModule  as htm 

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

tipIds = [4, 8, 12, 16, 20]



def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear_aspect_ratio = (A + B) / (2.0 * C)
    return ear_aspect_ratio

def get_className(classNo):
    p = []
    for i in os.listdir(r'C:\Users\Rajesh\Desktop\Project\open_cv\images'):
        p.append(i)
    return p[classNo]

doubt=0
dict1={}
for i in os.listdir(r'C:\Users\Rajesh\Desktop\Project\open_cv\images'):
    dict1[i]=0


wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
font = cv2.FONT_HERSHEY_COMPLEX

facedetect = cv2.CascadeClassifier(r"C:\Users\Rajesh\Desktop\Project\open_cv\haar_face.xml")

model = cv2.face.LBPHFaceRecognizer_create()
model.read(r'C:\Users\Rajesh\Desktop\Project\open_cv\face_trained2.yml')

hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor(r"C:\Users\Rajesh\Desktop\Project\open_cv\shape_predictor_68_face_landmarks.dat")


In [ ]:

from flask import Flask, render_template
from flask_socketio import SocketIO
from flask_cors import CORS
import random

app = Flask(__name__)
CORS(app, origins=["http://localhost:58015"])  
socketio = SocketIO(app)






@app.route('/')
def index():
    return render_template('index.html')

def data_generator():
    global dict1 
    global doubt
    while True:
        finger_counts = []
        drowsy_list = []
        success, imgOrignal = cap.read()
        img = cv2.cvtColor(imgOrignal, cv2.COLOR_BGR2RGB)
        faces = facedetect.detectMultiScale(imgOrignal, 1.3,5)
        for x, y, w, h in faces:
            crop_img = imgOrignal[y:y + h, x:x + h]

            crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
            cropped = (imgOrignal[:, x-h:x + h + h//2]).copy()

            classIndex, probabilityValue = model.predict(crop_img)

            if classIndex >= 0:
                cv2.rectangle(imgOrignal, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.rectangle(imgOrignal, (x, y - 40), (x + w, y), (0, 255, 0), -2)
                cv2.putText(imgOrignal, str(get_className(classIndex)), (x, y - 10), font, 0.75, (0, 0, 255), 1,
                            cv2.LINE_AA)
                faces = hog_face_detector(crop_img)
                for face in faces:
                    face_landmarks = dlib_facelandmark(crop_img, face)
                    leftEye = []
                    rightEye = []

                    for n in range(36, 42):
                        x = face_landmarks.part(n).x
                        y = face_landmarks.part(n).y
                        leftEye.append((x, y))
                        next_point = n + 1
                        if n == 41:
                            next_point = 36
                        x2 = face_landmarks.part(next_point).x
                        y2 = face_landmarks.part(next_point).y
                        cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

                    for n in range(42, 48):
                        x = face_landmarks.part(n).x
                        y = face_landmarks.part(n).y
                        rightEye.append((x, y))
                        next_point = n + 1
                        if n == 47:
                            next_point = 42
                        x2 = face_landmarks.part(next_point).x
                        y2 = face_landmarks.part(next_point).y
                        cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

                    left_ear = calculate_EAR(leftEye)
                    right_ear = calculate_EAR(rightEye)

                    EAR = (left_ear + right_ear) / 2
                    EAR = round(EAR, 2)

                    if EAR < 0.19:
                        cv2.putText(crop_img, "DROWSY", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
                        cv2.putText(crop_img, "Are you Sleepy?", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2,
                                    (0, 0, 255), 4)
                        drowsy_list.append(get_className(classIndex))

            cv2.imshow("Cropped",cropped)
            cv2.imshow("Drowsy", crop_img)
        cv2.imshow("Face Recognition", imgOrignal)
        
        for j in drowsy_list :
            (dict1[j]) += 1

        for i,j in dict1.items():
            if j > 3 :
                socketio.emit('data', {'data': i}, namespace='/drowsy')
                socketio.sleep(1)
                dict1[i] =0  

        key = cv2.waitKey(1)
        if key == 27:
            break
        

    cap.release()
    cv2.destroyAllWindows()        
            
@socketio.on('connect', namespace='/drowsy')
def handle_connect():
    socketio.start_background_task(data_generator)

In [ ]:
if __name__ == "__main__":
    socketio.run(app,port=5001,allow_unsafe_werkzeug=True)